In [ ]:
# Imports:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [ ]:
# Define excel path

excelPaths = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/presentations/update_slides')
excelPath = excelPaths.joinpath('2023C3_NEXAFS_summary.xlsx')
outPath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/data_analysis/rsoxs_suite/nexafs_plots/2023C3/bar_plot_tilt_summaries')

In [ ]:
# Function to determine the color and alpha based on tilt angle and quality
def get_bar_properties(tilt_angle, quality):
    # Determine the color
    color = 'red' if tilt_angle <= 55 else 'blue'
    
    # Determine the alpha (opacity)
    alpha = quality / 5.0  # Since 5 is the highest quality
    
    return color, alpha

In [ ]:
# df = pd.read_excel(excelPath, sheet_name='Carbon NEXAFS')
df = pd.read_excel(excelPath, sheet_name='Nitrogen NEXAFS')

display(df)

In [ ]:
# Modify the list comprehension for new_columns
new_columns = []
for i in range(df.shape[1]):
    if str(df.iloc[0, i]) == 'nan':
        # When it's nan, repeat the last non-nan value (if there is one)
        new_columns.append(str(df.iloc[0, i-1]) + ' ' + str(df.iloc[1, i]))
    elif str(df.iloc[0,i]) == 'Sample Name':
        new_columns.append(str(df.iloc[0,i]))
    else:
        new_columns.append(str(df.iloc[0, i]) + ' ' + str(df.iloc[1, i]))

# Assign the new column names
df.columns = new_columns

# Drop the two rows you used for the column names
df = df.drop([0, 1]).reset_index(drop=True)

display(df)
# new_columns

In [ ]:
condition = ~(df['RSoXS Chamber TEY Quality (0-5)'].isna() & 
              df['NEXAFS Chamber TEY Quality (0-5)'].isna() & 
              df['NEXAFS Chamber PEY Quality (0-5)'].isna() & 
             df['NEXAFS Chamber FY Quality (0-5)'].isna())
condition

df_filtered = df[condition]
df_filtered

In [ ]:
# Carbon
# df_groups = [[0, 3, 5, 6],
#              [8, 10],
#              [12, 13, 16, 17],
#              [20, 24], 
#              [26, 27, 28, 29],
#              [30, 31, 32, 33], 
#              [34, 35, 36, 37],
#              [38, 39, 40, 41],
#              [42, 43, 44, 45]]

# Nitrogen
df_groups = [[0, 3, 5, 6], 
             [8, 10],
             [12, 13, 16, 17],
             [18, 20, 24],
             [30, 32],
             [42, 43, 44, 45]]

for grouping in df_groups:
    # df_group = df_filtered.loc[grouping]
    df_group = df.loc[grouping]
    display(df_group)

In [ ]:
# Plotting for nitrogen

for peak_index, peak_name in enumerate(['Peak A', 'Peak B', 'Peak C']):
    for grouping in df_groups:
        df_group = df.loc[grouping]
        composition = df_group['Sample Name'].iloc[0].split(' ')[0]
        solvents = []
        for sample_name in df_group['Sample Name']:
            solvent = sample_name.split(' ')[1]
            solvents.append(solvent)

        # Set up the figure and axis
        fig, ax = plt.subplots(figsize=(7, 3.5))
        fig.set(tight_layout=True, dpi=120)
        fig.suptitle(f'Nitrogen NEXAFS {peak_name} Molecular Tilt Summary for {composition}', y=0.93, x=0.54)

        # Bar width and positions
        bar_width = 0.15
        ind = np.arange(len(df_group))  # Indexes for sample names

        measurements = [
            ('RSoXS Chamber TRMSN Tilt Angles (α) [°]', 'RSoXS Chamber TRMSN Quality (0-5)'),
            ('RSoXS Chamber TEY Tilt Angles (α) [°]', 'RSoXS Chamber TEY Quality (0-5)'),
            ('NEXAFS Chamber TEY Tilt Angles (α) [°]', 'NEXAFS Chamber TEY Quality (0-5)'),
            ('NEXAFS Chamber PEY Tilt Angles (α) [°]', 'NEXAFS Chamber PEY Quality (0-5)'),
            ('NEXAFS Chamber FY Tilt Angles (α) [°]', 'NEXAFS Chamber FY Quality (0-5)')
        ]

        annotation_key = {'RSoXS Chamber TRMSN Tilt Angles (α) [°]': 'RS TRMSN',
                          'RSoXS Chamber TEY Tilt Angles (α) [°]': 'RS TEY', 
                          'NEXAFS Chamber TEY Tilt Angles (α) [°]': 'NF TEY', 
                          'NEXAFS Chamber PEY Tilt Angles (α) [°]': 'NF PEY', 
                          'NEXAFS Chamber FY Tilt Angles (α) [°]': 'NF FY'}

        # Iterate over each measurement
        for i, (angle_col, quality_col) in enumerate(measurements):
            # List to store bar properties
            heights = []
            colors = []
            alphas = []

            for index, row in df_group.iterrows():
                quality = row[quality_col]

                if str(row[angle_col])=='nan':
                    heights.append(0)
                    colors.append('white')
                    alphas.append(0)       
                else:
                    tilt_angle = ast.literal_eval(row[angle_col])[peak_index]
                    color, alpha = get_bar_properties(tilt_angle, quality)
                    height = 55 - tilt_angle

                    heights.append(height)
                    colors.append(color)
                    alphas.append(alpha)

            # Plot the bars for this measurement
            for sample_index, (height, color, alpha) in enumerate(zip(heights, colors, alphas)):
                ax.bar(sample_index + i*bar_width, height, bar_width, color=color, alpha=alpha, edgecolor="white", linewidth=1)
                # ax.annotate(' '.join(angle_col.split(' ')[0:3]), 
                ax.annotate(annotation_key[angle_col], 
                            xy=(sample_index + i*bar_width, 0), 
                            xytext=(2,5), 
                            textcoords="offset points", 
                            ha='center', va='bottom', 
                            rotation=90)        

        # Position the x-axis at y=0 which corresponds to 55 degrees in our plot
        ax.axhline(0, color='black')  # Adds the x-axis at the value of 55

        # Set x-ticks to be the sample names, adjusting position to center them amongst the bars
        ax.set_xticks(ind + bar_width*2)
        # ax.set_xticklabels(df_group['Sample Name'], rotation=45, ha="right")
        # ax.set_xticklabels(solvents, rotation=0, ha="right")
        ax.set_xticklabels(solvents)



        # Optional: You can also set the y-tick labels to indicate the actual tilt angles
        # current_y_ticks = ax.get_yticks()
        # ax.set_yticklabels([int(55 + y_val) for y_val in current_y_ticks])

        ax.set(ylabel=r'Molecular Tilt: 55 - $\alpha$ [°]', ylim=(-15, 40), xlabel='Solvent')
        plt.savefig(outPath.joinpath('nitrogen', f'{composition}_nitrogen_{peak_name}_nexafs_summary.png'), dpi=150)

        plt.show()
        # fname = f'{composition}_carbon_nexafs_summary'
        plt.close('all')

In [ ]:
outPath

In [ ]:
# Plotting for carbon

for grouping in df_groups:
    df_group = df_filtered.loc[grouping]
    composition = df_group['Sample Name'].iloc[0].split(' ')[0]
    solvents = []
    for sample_name in df_group['Sample Name']:
        solvent = sample_name.split(' ')[1]
        solvents.append(solvent)
        
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(7, 3.5))
    fig.set(tight_layout=True, dpi=120)
    fig.suptitle(f'Carbon NEXAFS Molecular Tilt Summary for {composition}', y=0.93)

    # Bar width and positions
    bar_width = 0.15
    ind = np.arange(len(df_group))  # Indexes for sample names

    measurements = [
        ('RSoXS Chamber TRMSN Tilt Angle (α) [°]', 'RSoXS Chamber TRMSN Quality (0-5)'),
        ('RSoXS Chamber TEY Tilt Angle (α) [°]', 'RSoXS Chamber TEY Quality (0-5)'),
        ('NEXAFS Chamber TEY Tilt Angle (α) [°]', 'NEXAFS Chamber TEY Quality (0-5)'),
        ('NEXAFS Chamber PEY Tilt Angle (α) [°]', 'NEXAFS Chamber PEY Quality (0-5)'),
        ('NEXAFS Chamber FY Tilt Angle (α) [°]', 'NEXAFS Chamber FY Quality (0-5)')
    ]

    annotation_key = {'RSoXS Chamber TRMSN Tilt Angle (α) [°]': 'RS TRMSN',
                      'RSoXS Chamber TEY Tilt Angle (α) [°]': 'RS TEY', 
                      'NEXAFS Chamber TEY Tilt Angle (α) [°]': 'NF TEY', 
                      'NEXAFS Chamber PEY Tilt Angle (α) [°]': 'NF PEY', 
                      'NEXAFS Chamber FY Tilt Angle (α) [°]': 'NF FY'}

    # Iterate over each measurement
    for i, (angle_col, quality_col) in enumerate(measurements):
        # List to store bar properties
        heights = []
        colors = []
        alphas = []

        for index, row in df_group.iterrows():
            tilt_angle = row[angle_col]
            # print(tilt_angle)
            quality = row[quality_col]

            if np.isnan(tilt_angle) or quality == 0:
                heights.append(0)
                colors.append('white')
                alphas.append(0)
            else:
                color, alpha = get_bar_properties(tilt_angle, quality)
                height = 55 - tilt_angle

                heights.append(height)
                colors.append(color)
                alphas.append(alpha)

        # Plot the bars for this measurement
        for sample_index, (height, color, alpha) in enumerate(zip(heights, colors, alphas)):
            ax.bar(sample_index + i*bar_width, height, bar_width, color=color, alpha=alpha, edgecolor="white", linewidth=1)
            # ax.annotate(' '.join(angle_col.split(' ')[0:3]), 
            ax.annotate(annotation_key[angle_col], 
                        xy=(sample_index + i*bar_width, 0), 
                        xytext=(2,5), 
                        textcoords="offset points", 
                        ha='center', va='bottom', 
                        rotation=90)        

    # Position the x-axis at y=0 which corresponds to 55 degrees in our plot
    ax.axhline(0, color='black')  # Adds the x-axis at the value of 55

    # Set x-ticks to be the sample names, adjusting position to center them amongst the bars
    ax.set_xticks(ind + bar_width*2)
    # ax.set_xticklabels(df_group['Sample Name'], rotation=45, ha="right")
    # ax.set_xticklabels(solvents, rotation=0, ha="right")
    ax.set_xticklabels(solvents)



    # Optional: You can also set the y-tick labels to indicate the actual tilt angles
    # current_y_ticks = ax.get_yticks()
    # ax.set_yticklabels([int(55 + y_val) for y_val in current_y_ticks])

    ax.set(ylabel=r'Molecular Tilt: 55 - $\alpha$ [°]', ylim=(-15, 40), xlabel='Solvent')
    plt.savefig(outPath.joinpath('carbon', f'{composition}_carbon_nexafs_summary.png'), dpi=150)

    plt.show()
    # fname = f'{composition}_carbon_nexafs_summary'
    plt.close('all')